epub is a hidden zip file; change epub ending to zip and unpack into a folder

In [ ]:
#may have to install packages with pip
# in Colab use !pip install bs4
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id})
        return data

# loop over all HTML files in a folder and extract information


In [ ]:
folder_path = 'bachmannfrisch' #name of the subfolder in the folder containing the script 
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0])):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)

In [ ]:
#not all of these cells are necessary and in the right order, this was used to work towards solving specific problems
folder_path = 'bachmannfrisch'
data = []
for file_id in sorted([int(re.findall(r'\d+', filename)[0]) for filename in os.listdir(folder_path) if filename.endswith('.html')]):
    filename = f"{file_id}.html"
    filepath = os.path.join(folder_path, filename)
    print(f"Processing {filepath}")
    data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)

In [ ]:
#here we worked to retrieve specific spans within the html, customize according to your own data
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+(?=\.)', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+(?=\.)', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
df.tail

In [ ]:
# Assuming that 'df' is the name of the DataFrame
row_index = 225 # The index of the row to display
column_name = 'date' # The name of the column to display

# Use .loc to select the row of interest and access the value of the column
text = df.loc[row_index, column_name]

# Print the complete text
print(text)

In [ ]:
for colName in df.columns:
    if colName == "date":
        df = df[ df[colName].notna() ]
    else:
        df = df[ df[colName].isna() ]



In [ ]:
print(df)

In [ ]:
df.to_csv('result.csv', encoding='utf-8') #writing the resulting dataframe to a csv file

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract date from <p class="brief_datum"> tag
                date_tag = soup.find('p', {'class': 'brief_datum'})
                if date_tag:
                    date_text = date_tag.text.strip()
                    # extract date using regex
                    match = re.search(r'(\d+[\s\.-]*\w*[\s\.-]*\d+)', date_text)
                    if match:
                        date_str = match.group().replace('.', '').replace('-', '').replace(' ', '.')
                        try:
                            date = datetime.strptime(date_str, '%d.%m.%Y').date()
                        except ValueError:
                            try:
                                date = datetime.strptime(date_str, '%d.%m.%y').date()
                            except ValueError:
                                try:
                                    date = datetime.strptime(date_str, '%d.%B.%Y').date()
                                except ValueError:
                                    date = 'NA'
                    else:
                        date = 'NA'
                else:
                    date = 'NA'
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'date': date})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
#for filename in sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0])):
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+(?=\.)', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)
        
# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True) 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract date from <p class="brief_datum"> tag
                date_tags = p_tag.find_all('p', {'class': 'brief_datum'})
                dates = []
                for date_tag in date_tags:
                    date_text = date_tag.text.strip()
                    # regex to match various date formats in German
                    match = re.search(r'\b(\d{1,2})([^\d]+)(\d{1,2})([^\d]+)(\d{2,4})\b', date_text)
                    if match:
                        date_str = match.group(0)
                        try:
                            # convert to datetime object
                            date = datetime.strptime(date_str, '%d.%m.%Y').date()
                            dates.append(date)
                        except ValueError:
                            pass
                    else:
                        dates.append(None)
                # create dictionary of data for this HTML file
                data_dict = {'id': id, 'title': title, 'letter_text': letter_text, 'file_id': int(re.search(r'\d+', filepath).group())}
                # add date to dictionary
                if len(dates) == 0:
                    data_dict['date'] = None
                elif len(dates) == 1:
                    data_dict['date'] = dates[0]
                else:
                    # choose the earliest date as the main date
                    data_dict['date'] = min(dates)
                data.append(data_dict)
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)



In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                
                # extract datum from <p> tag
                datum_tags = p_tag.find_next_siblings('p', {'class': 'brief_datum'})
                datum_values = []
                for datum_tag in datum_tags:
                    datum_text = datum_tag.text.strip()
                    if re.match(r'^\d{1,2}\.?\s*[-.,/\\]?\s*\d{1,2}\.?\s*[-.,/\\]?\s*\d{2,4}$', datum_text):
                        try:
                            datum_value = datetime.strptime(datum_text, '%d.%m.%Y').date()
                            datum_values.append(datum_value)
                        except ValueError:
                            pass
                    elif re.match(r'^\d{1,2}\s*[-.,/\\]?\s*\w+\s+\d{4}$', datum_text):
                        try:
                            datum_value = datetime.strptime(datum_text, '%d. %B %Y').date()
                            datum_values.append(datum_value)
                        except ValueError:
                            pass
                    elif re.match(r'^[A-Za-z]+\s*,?\s*\d{1,2}\.?\s*[-.,/\\]?\s*\d{1,2}\.?\s*[-.,/\\]?\s*\d{2,4}$', datum_text):
                        datum_value = re.search(r'\d{1,2}\.?\s*[-.,/\\]?\s*\d{1,2}\.?\s*[-.,/\\]?\s*\d{2,4}', datum_text).group(0)
                        try:
                            datum_value = datetime.strptime(datum_value, '%d.%m.%Y').date()
                            datum_values.append(datum_value)
                        except ValueError:
                            pass
                if datum_values:
                    datum = datum_values[0]
                else:
                    datum = "NA"
                
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'datum': datum})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
#for filename in sorted(os.listdir(folder_path), key=lambda x:
# loop over all HTML files in a folder and extract information
#folder_path = 'bachmannfrisch'
#data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)

In [ ]:
print(df)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from <p> tag
                datum_tags = soup.find_all('p', {'class': 'brief_datum'})
                datum = None
                for datum_tag in datum_tags:
                    datum_text = datum_tag.get_text().strip()
                    # extract date from text using regex
                    match = re.search(r'(\d{1,2})[\.\s-]+([\wäöü]+)[\s-]+(\d{2,4})', datum_text)
                    if match:
                        # convert to date object
                        try:
                            datum = datetime.strptime(match.group(), '%d. %B %Y').date()
                        except ValueError:
                            try:
                                datum = datetime.strptime(match.group(), '%d.%m.%Y').date()
                            except ValueError:
                                try:
                                    datum = datetime.strptime(match.group(), '%d-%m-%Y').date()
                                except ValueError:
                                    pass
                        break
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'datum': datum})
                print(f"Processed id {id}: title='{title}', letter_text='{letter_text}', file_id={file_id}, datum={datum}")
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                
                # extract date from <p> tag
                datum_tags = p_tag.find_all('p', {'class': 'brief_datum'})
                date = None
                for datum_tag in datum_tags:
                    datum_str = datum_tag.text.strip()
                    # match various date formats in German
                    match = re.search(r'(\d{1,2}\s*[.-]\s*\w+\s*[.-]\s*\d{2,4}|\d{1,2}\s*[.-]\s*\d{1,2}\s*[.-]\s*\d{2,4}|\w+\s+\d{1,2},\s*\d{2,4})', datum_str)
                    if match:
                        date_str = match.group()
                        try:
                            date = datetime.strptime(date_str, '%d.%m.%Y').date()
                        except ValueError:
                            try:
                                date = datetime.strptime(date_str, '%d-%m-%Y').date()
                            except ValueError:
                                try:
                                    date = datetime.strptime(date_str, '%d %b %Y').date()
                                except ValueError:
                                    pass
                    if date:
                        break
                
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'date': date})
        return data

# loop over all
# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from dateutil.parser import parse

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from <p> tag
                datum_tags = p_tag.find_all('p', {'class': 'brief_datum'})
                date = None
                for datum_tag in datum_tags:
                    datum_text = datum_tag.text.strip()
                    try:
                        date = parse(datum_text, dayfirst=True)
                    except ValueError:
                        pass
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'date': date, 'file_id': file_id})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from dateutil.parser import parse

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None

                # extract date from <p class="brief_datum"> tag
                datum_tags = p_tag.find_all('p', {'class': 'brief_datum'})
                datum = None
                if datum_tags:
                    for tag in datum_tags:
                        # Check if the text inside the tag resembles a date
                        try:
                            parse(tag.text.strip(), dayfirst=True, yearfirst=False)
                        except ValueError:
                            continue
                        datum = tag.text.strip()
                        break

                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'datum': datum})

        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
print(df)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract brief_datum from <p> tag most resembling a date
                datum_tags = div_tag.find_all('p', {'class': 'brief_datum'})
                if datum_tags:
                    datum_text = ""
                    for datum_tag in datum_tags:
                        datum_text = datum_tag.text.strip()
                        if re.match(r'\d+\.\s*[\d\w]+\s*\d+', datum_text):
                            break
                    else:
                        datum_text = "NA"
                else:
                    datum_text = "NA"
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'date': datum_text})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
df.head

In [ ]:
for colName in df.columns:
    if colName == "date":
        df = df[ df[colName].notna() ]
    else:
        df = df[ df[colName].isna() ]

In [ ]:
df.head

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from brief_datum tag
                datum_tags = div_tag.find_all('p', {'class': 'brief_datum'})
                if datum_tags:
                    datum_text = datum_tags[-1].text.strip()
                    if re.match(r'\d+\.\s*\w+\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d. %B %Y')
                    elif re.match(r'\d+\s*-\s*\d+\s*-\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d - %m - %Y')
                    elif re.match(r'\d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d.%m.%Y')
                    elif re.match(r'\w+, \d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(re.findall(r'\d+\.\s*\d+\.\s*\d+', datum_text)[0], '%d.%m.%Y')
                    else:
                        datum = 'NA'
                else:
                    datum = 'NA'
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'date': datum})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from brief_datum tag
                datum_tags = div_tag.find_all('p', {'class': 'brief_datum'})
                if datum_tags:
                    datum_text = datum_tags[-1].text.strip().replace('\xa0', ' ')
                    if re.match(r'\d+\.\s*\w+\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d. %B %Y')
                    elif re.match(r'\d+\s*-\s*\d+\s*-\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d - %m - %Y')
                    elif re.match(r'\d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d.%m.%Y')
                    elif re.match(r'\w+, \d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(re.findall(r'\d+\.\s*\d+\.\s*\d+', datum_text)[0], '%d.%m.%Y')
                    else:
                        datum = 'NA'
                else:
                    datum = 'NA'
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'date': datum})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from brief_datum tag
                datum_tags = div_tag.find_all('p', {'class': 'brief_datum'})
                if datum_tags:
                    datum_text = datum_tags[-1].text.strip().replace('\xa0', ' ')
                    if re.match(r'\d+\.\s*\w+\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d. %B %Y')
                    elif re.match(r'\d+\s*-\s*\d+\s*-\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d - %m - %Y')
                    elif re.match(r'\d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d.%m.%Y')
                    elif re.match(r'\w+, \d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(re.findall(r'\d+\.\s*\d+\.\s*\d+', datum_text)[0], '%d.%m.%Y')
                    else:
                        datum = 'NA'
                else:
                    datum = 'NA'
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'file_id': file_id, 'date': datum})
        return data




In [ ]:
# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)

In [ ]:
#change locale if colab moans about this
import locale
#locale.setlocale(locale.LC_TIME, 'de_DE')
locale.setlocale(locale.LC_TIME, 'de_DE.utf8')
# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
df.drop(columns=['file_id'], inplace=True)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime

# define a function to extract information from each HTML file
def extract_info(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        # extract id and title from <p> tag
        p_tags = soup.find_all('p', {'class': 'brief_titel1'})
        data = []
        for p_tag in p_tags:
            id = p_tag.get('id')
            title = p_tag.get('title')
            # extract letter text from <div> tag
            div_tag = p_tag.find_next_sibling('div', {'class': 'letter'})
            if div_tag:
                letter_text = ''
                for p_tag in div_tag.find_all('p'):
                    if 'class' in p_tag.attrs and ('gt' in p_tag['class'] or 'brief_gt' in p_tag['class']):
                        letter_text += p_tag.text.strip().replace('</p>', ' ') + ' '
                # extract numeric portion of filename using regex
                match = re.search(r'\d+', filepath)
                if match:
                    file_id = int(match.group())
                else:
                    file_id = None
                # extract date from brief_datum tag
                datum_tags = div_tag.find_all('p', {'class': 'brief_datum'})
                if datum_tags:
                    datum_text = datum_tags[-1].text.strip().replace('\xa0', ' ')
                    # convert date string to datetime object
                    if re.match(r'\d+\.\s*\w+\s*\d+', datum_text):
                        try:
                            datum = datetime.strptime(datum_text, '%d. %B %Y')
                        except ValueError:
                            try:
                                datum = datetime.strptime(datum_text, '%d. %b %Y')
                            except ValueError:
                                datum = 'NA'
                    elif re.match(r'\d+\s*-\s*\d+\s*-\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d - %m - %Y')
                    elif re.match(r'\d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(datum_text, '%d.%m.%Y')
                    elif re.match(r'\w+, \d+\.\s*\d+\.\s*\d+', datum_text):
                        datum = datetime.strptime(re.findall(r'\d+\.\s*\d+\.\s*\d+', datum_text)[0], '%d.%m.%Y')
                    else:
                        datum = 'NA'
                else:
                    datum = 'NA'
                data.append({'id': id, 'title': title, 'letter_text': letter_text, 'date': datum})
        return data

# loop over all HTML files in a folder and extract information
folder_path = 'bachmannfrisch'
data = []
for filename in sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group())):
    if filename.endswith('.html'):
        filepath = os.path.join(folder_path, filename)
        print(f"Processing {filepath}")
        data += extract_info(filepath)

# create a Pandas DataFrame from the extracted data
df = pd.DataFrame(data)
#df.drop(columns=['file_id'], inplace=True)


In [ ]:
print(df)